# Simple visualization of each item

This notebook is a simple visualization of each item.

It may be useful for grasping the whole.

![img1_1](https://raw.githubusercontent.com/Lux-AI-Challenge/Lux-Design-2021/master/assets/game_board.png)

In [ ]:
!pip install kaggle-environments -U -q
!cp -r ../input/lux-ai-2021/* .

In [ ]:
# copied from https://www.kaggle.com/ilialar/lux-ai-risk-averse-baseline
# %%writefile agent.py
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import numpy as np

### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile, closest_dist

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    closest_dist = math.inf
    if len(player.cities) > 0:
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile, closest_dist

game_state = None

def get_random_step():
    return np.random.choice(['s','n','w','e'])


def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
        
    # max number of units available
    units_cap = sum([len(x.citytiles) for x in player.cities.values()])
    # current number of units
    units  = len(player.units)
    
    cities = list(player.cities.values())
    if len(cities) > 0:
        city = cities[0]
        created_worker = (units >= units_cap)
        for city_tile in city.citytiles[::-1]:
            if city_tile.can_act():
                if created_worker:
                    # let's do research
                    action = city_tile.research()
                    actions.append(action)
                else:
                    # let's create one more unit in the last created city tile if we can
                    action = city_tile.build_worker()
                    actions.append(action)
                    created_worker = True
    
    
    # we want to build new tiless only if we have a lot of fuel in all cities
    can_build = True
    night_steps_left = ((359 - observation["step"]) // 40 + 1) * 10
    for city in player.cities.values():            
        if city.fuel / (city.get_light_upkeep() + 20) < min(night_steps_left, 30):
            can_build = False
       
    steps_until_night = 30 - observation["step"] % 40
    
    
    # we will keet all tiles where any unit wants to move in this set to avoid collisions
    taken_tiles = set()
    for unit in player.units:
        # it is too strict but we don't allow to go to the the currently occupied tile
        taken_tiles.add((unit.pos.x, unit.pos.y))
        
    for city in opponent.cities.values():
        for city_tile in city.citytiles:
            taken_tiles.add((city_tile.pos.x, city_tile.pos.y))
    
    # we can collide in cities so we will use this tiles as exceptions
    city_tiles = {(tile.pos.x, tile.pos.y) for city in player.cities.values() for tile in city.citytiles}
    
    
    for unit in player.units:
        if unit.can_act():
            closest_resource_tile, closest_resource_dist = find_closest_resources(unit.pos, player, resource_tiles)
            closest_city_tile, closest_city_dist = find_closest_city_tile(unit.pos, player)
            
            # we will keep possible actions in a priority order here
            directions = []
            
            # if we can build and we are near the city let's do it
            if unit.is_worker() and unit.can_build(game_state.map) and ((closest_city_dist == 1 and can_build) or (closest_city_dist is None)):
                # build a new cityTile
                action = unit.build_city()
                actions.append(action)  
                can_build = False
                continue
            
            # base cooldown for different units types
            base_cd = 2 if unit.is_worker() else 3
            
            # how many steps the unit needs to get back to the city before night (without roads)
            steps_to_city = unit.cooldown + base_cd * closest_city_dist
            
            # if we are far from the city in the evening or just full let's go home
            if (steps_to_city + 3 > steps_until_night or unit.get_cargo_space_left() == 0) and closest_city_tile is not None:
                actions.append(annotate.line(unit.pos.x, unit.pos.y, closest_city_tile.pos.x, closest_city_tile.pos.y))
                directions = [unit.pos.direction_to(closest_city_tile.pos)]
            else:
                # if there is no risks and we are not mining resources right now let's move toward resources
                if closest_resource_dist != 0 and closest_resource_tile is not None:
                    actions.append(annotate.line(unit.pos.x, unit.pos.y, closest_resource_tile.pos.x, closest_resource_tile.pos.y))
                    directions = [unit.pos.direction_to(closest_resource_tile.pos)]
                    # optionally we can add random steps
                    for _ in range(2):
                        directions.append(get_random_step())

            moved = False
            for next_step_direction in directions:
                next_step_position = unit.pos.translate(next_step_direction, 1)
                next_step_coordinates = (next_step_position.x, next_step_position.y)
                # make only moves without collision
                if next_step_coordinates not in taken_tiles or next_step_coordinates in city_tiles:
                    action = unit.move(next_step_direction)
                    actions.append(action)
                    taken_tiles.add(next_step_coordinates)
                    moved = True
                    break
            
            if not moved:
                # if we are not moving the tile is occupied
                taken_tiles.add((unit.pos.x,unit.pos.y))
    return actions

In [ ]:
from kaggle_environments import make
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
env = make("lux_ai_2021", configuration={"loglevel": 0}, debug=True)
steps = env.run([agent, "simple_agent"])

steps_lst = []
for steps_players in steps:
    for i in steps_players:
        steps_lst.append(pd.io.json.json_normalize(i))

steps_df = pd.concat(steps_lst).reset_index(drop=True)
player_steps_df = steps_df.loc[steps_df['observation.step'].notnull()]
opp_steps_df = steps_df.loc[steps_df['observation.step'].isnull()]

plt.plot(player_steps_df['observation.reward'])
plt.plot(opp_steps_df['observation.reward'])
plt.title('reward timeline')
plt.legend(['player','opponent'])

~~# Observe the specified step~~

In [ ]:
# obs_step = 300#200
# player_actions = player_steps_df.action.tolist()[1:]
# opp_actions = opp_steps_df.action.tolist()[1:]
# env.reset()
# for i in range(obs_step):
#     steps = env.step([player_actions[i], opp_actions[i]])    
env.render(mode="ipython", width=900, height=600)

In [ ]:
player_id = 0
player = game_state.players[player_id]
opponent = game_state.players[(player_id + 1) % 2]

bd = game_state.map.map

resource_maps ={'wood':1,
                'coal':2,
                'uranium':3}

height, width = game_state.map_height, game_state.map_width 
bd_res_type = np.zeros([height, width],np.int8)
bd_wood = np.zeros([height, width],np.int16)
bd_coal = np.zeros([height, width],np.int16)
bd_uranium = np.zeros([height, width],np.int16)
bd_res_amount = np.zeros([height, width],np.int16)
bd_cityid = np.full([height, width],-1, np.int8)
bd_city_cooldown = np.full([height, width], -1, np.int8)
bd_city_can_act =  np.full([height, width],-1, np.int8)
bd_city =  np.full([height, width], -1, np.int8)
bd_road = np.full([height, width], -1, np.float32)#upgrade per 0.5
#bd_city_fuel = np.full([height, width], -1, np.float32)
#bd_city_light_upkeep = np.full([height, width], -1, np.float32)

bd_unit = np.full([height, width],-1, np.int8)

for y in range(height):
    for x in range(width):
        if bd[y][x].has_resource():
            bd_res_type[y][x] = resource_maps[bd[y][x].resource.type]
            bd_res_amount[y][x] = bd[y][x].resource.amount
            if bd[y][x].resource.type=='wood':
                bd_wood[y][x] = bd[y][x].resource.amount
            elif bd[y][x].resource.type=='coal':
                bd_coal[y][x] = bd[y][x].resource.amount
            elif bd[y][x].resource.type=='uranium':
                bd_uranium[y][x] = bd[y][x].resource.amount

                
        if bd[y][x].citytile is not None:
            bd_cityid[y][x] = int(bd[y][x].citytile.cityid.split('_')[-1])#bd[y][x]citytile.cityid#
            bd_city_cooldown[y][x] = bd[y][x].citytile.cooldown
            bd_city_can_act[y][x] = bd[y][x].citytile.can_act()
            bd_city[y][x] = bd[y][x].citytile.team
            #try:
            #    city = player.cities[bd[y][x].citytile.cityid]
            #except:
            #    city = opponent.cities[bd[y][x].citytile.cityid]
            #bd_city_fuel[y][x] = city.fuel
            #bd_city_light_upkeep[y][x] = city.light_upkeep
        bd_road[y][x]= bd[y][x].road
        
for unit in player.units:
    bd_unit[unit.pos.y][unit.pos.x] = 0
for unit in opponent.units:
    bd_unit[unit.pos.y][unit.pos.x] = 1
    
    
lst = []
for unit in player.units+opponent.units:
    lst.append(
        [unit.id,
         unit.team,
         unit.pos.x,
         unit.pos.y,
         'worker' if unit.is_worker() else 'cart',
         unit.cooldown,
         unit.cargo.wood,
         unit.cargo.coal,
         unit.cargo.uranium,
         unit.get_cargo_space_left(),
         unit.can_act(),
        ]
    )
    
unit_df = pd.DataFrame(lst)
unit_df.columns = ['id','team','pos_x','pos_y','type','cooldown',
                   'cargo.wood','cargo.coal','cargo.uranium',
                  'cargo_space_left','can_act']

lst = []
cities = dict(player.cities, **opponent.cities)
for city_id in cities.keys():
    lst.append(
        [cities[city_id].cityid,
         cities[city_id].team,
         cities[city_id].fuel, 
         cities[city_id].get_light_upkeep(),])
    
city_df = pd.DataFrame(lst, columns=['cityid','team','fuel','light_upkeep'])



lst = []
resource_tiles = find_resources(game_state)
for cell in resource_tiles:
    lst.append([
        cell.resource.type, 
        cell.resource.amount,
        cell.pos.x,
        cell.pos.y,
        ])

resource_tiles_df = pd.DataFrame(lst)
resource_tiles_df.columns = ['type','amount','pos_x','pos_y',]

lst = []
for team in [player, opponent]:
    lst.append(
        [team.team,
         team.city_tile_count,
         len(team.units),
         team.research_points,
         team.researched_coal(),
         team.researched_uranium(),
        ])
    
team_df = pd.DataFrame(lst, 
                       columns = ['team','city_tile_count', 'city_unit_count',
                                  'research_points','researched_coal','researched_uranium'])

lst = []
cities = dict(player.cities, **opponent.cities)
for city_id in cities.keys():
    for citytile in cities[city_id].citytiles:
        lst.append(
            [citytile.cityid,
             citytile.team,
             citytile.pos.x,
             citytile.pos.y,
             citytile.cooldown,
             citytile.can_act()])
citytile_df = pd.DataFrame(lst, columns=['cityid','team','pos_x','pos_y','cooldown','can_act'])

In [ ]:
!mkdir ./templates -p

In [ ]:
%%file templates/luxhtml.tpl

{% extends "html.tpl" %}
{% block table %}
<h1 class='table_title' style="background-color:#e7e7e7; text-align: center">{{ table_title|default("")}}</h1>
{{ super() }}
{% endblock table %}

In [ ]:
# https://pandas.pydata.org/pandas-docs/version/0.23/style.html
from jinja2 import Environment, ChoiceLoader, FileSystemLoader
from pandas.io.formats.style import Styler
from IPython.display import display, HTML
pd.options.display.max_columns = 32
class LuxStyler(Styler):
    env = Environment(
        loader=ChoiceLoader([
            FileSystemLoader("templates"),  # contains ours
            Styler.loader,  # the default
        ])
    )
    template = env.get_template("luxhtml.tpl")

def visualize(bd, title, default_value=-1):
    df = pd.DataFrame(bd).astype(str).replace(str(default_value),'')
    df.columns = [f'x_{s}' for s in df.columns]
    df.index = [f'y_{s}' for s in df.index]
    #display(df.astype(str).replace(str(default_value),'').style.set_caption(title))
    return LuxStyler(df).render(table_title=title)

def visualize_res_type(bd, title, default_value=-1):
    inv_resource_maps = {1:'w',2:'c',3:'u'}
    df = (pd.DataFrame(bd).replace(inv_resource_maps).astype(str)
                          .replace(str(default_value),'')
                          .replace({str(default_value),''}))
    df.columns = [f'x_{s}' for s in df.columns]
    df.index = [f'y_{s}' for s in df.index]
    return LuxStyler(df).render(table_title=title)

def visualize_cityid(bd, title):
    df = pd.DataFrame(bd_cityid).astype(str).applymap(lambda s:f'c_{s}').replace('c_-1','')
    df.columns = [f'x_{s}' for s in df.columns]
    df.index = [f'y_{s}' for s in df.index]
    return LuxStyler(df).render(table_title=title)


# Road Level

In [ ]:
HTML(visualize(bd_road, 'Road Level', default_value=0.))

# CityTiles info

In [ ]:
HTML(visualize(bd_city_cooldown, 'CityTile Cooldown'))

In [ ]:
HTML(visualize(bd_city_can_act, 'CityTile can_action', default_value=-1))

#### citytiles list

In [ ]:
citytile_df

# Cities info

In [ ]:
HTML(visualize_cityid(bd_cityid, 'City id'))

In [ ]:
HTML(visualize(bd_city, 'City Team'))


In [ ]:
#HTML(visualize(bd_city_fuel, 'City Fuel', default_value=-1.))

In [ ]:
#HTML(visualize(bd_city_light_upkeep, 'light_upkeep', default_value=-1.))

#### cities list

In [ ]:
city_df

# Resource

- w:wood
- c:coal
- u uranium

In [ ]:
HTML(visualize_res_type(bd_res_type, 'Resource Type', default_value=0))

In [ ]:
HTML(visualize_res_type(bd_wood, 'Resource Wood', default_value=0))

In [ ]:
HTML(visualize_res_type(bd_coal, 'Resource Coal', default_value=0))

In [ ]:
HTML(visualize_res_type(bd_uranium, 'Resource Uranium', default_value=0))

#### resources list

In [ ]:
resource_tiles_df

# Units

Since units can be stacked on a cell, teamid (player: 0, opponent: 1) is set if there is even one unit on the cell.

In [ ]:
HTML(visualize(bd_unit, 'Unit Team', default_value=-1))

#### Units list   

In [ ]:
unit_df

# Players

#### players list

In [ ]:
team_df